In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

In [2]:
env = gym.make('StockTradingEnvironment-v0', use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
])

In [3]:
pd.read_csv('src/environments/continuous/stock_trading/datasets/67b906a1ca1d5e7e03dd20f0c89fd03c/targets.csv').targets.iloc[0]

6.936458988189692

In [4]:
env.train_dataframe_id_range,env.test_dataframe_id_range

([0, 315], [316, 395])

In [5]:
env.load_dataset_by_index(0)
env.df

,open,high,low,close,volume,day,vix,turbulence,macd,boll_ub,...,neg_title,neu_title,pos_title,compound_title,neg_body,neu_body,pos_body,compound_body,aapl,fear greed
date,,,,,,,,,,,,,,,,,,,,,
2018-01-08,43.587502,43.902500,43.482498,41.573772,82271200,0,0.0952,0.061082,-0.003477,41.870444,...,0.000000,1.000000,0.000000,0.000000,0.008000,0.934000,0.058000,0.963700,0.395833,0.75
2018-01-09,43.637501,43.764999,43.352501,41.569000,86336000,1,0.1008,0.060920,-0.004579,41.805620,...,0.000000,0.946800,0.053200,0.088080,0.026400,0.919000,0.055000,0.684980,0.397083,0.75
2018-01-11,43.647499,43.872501,43.622501,41.795528,74670800,3,0.0988,0.061166,0.003795,41.864027,...,0.244000,0.756000,0.000000,-0.296525,0.041000,0.863750,0.095250,0.636025,0.379167,0.77
2018-01-12,44.044998,44.340000,43.912498,42.227123,101672400,4,0.1016,0.071771,0.027654,42.256032,...,0.000000,1.000000,0.000000,0.000000,0.126000,0.874000,0.000000,-0.893400,0.371667,0.79
2018-01-16,44.474998,44.847500,44.035000,42.012512,118263600,1,0.1166,0.177656,0.032756,42.292433,...,0.000000,0.931667,0.068333,0.067433,0.030000,0.849333,0.120667,0.729800,0.378333,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-30,40.532501,41.814999,40.459999,39.566917,169709600,0,0.1593,0.219233,-0.531165,43.128501,...,0.000000,1.000000,0.000000,0.000000,0.022500,0.877500,0.100500,0.939450,0.418333,0.42
2018-05-01,41.602501,42.299999,41.317501,40.486298,214277600,1,0.1549,0.087350,-0.454444,43.132534,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.962000,0.038000,0.381800,0.450417,0.38
2018-05-02,43.807499,44.437500,43.450001,42.274780,266157600,2,0.1597,0.129608,-0.246744,43.287283,...,0.040364,0.826455,0.133182,0.110127,0.063545,0.853364,0.083091,0.339791,0.431250,0.39


In [6]:
env.success_threshold

0.12262316637777877

In [7]:
import pandas as pd
episode_targets = []
for i in range(env.n_dataframes):
    env.load_dataset_by_index(i)
    raw_slice = env.df 
    episode_targets.append([
                    env.get_episode_target(raw_slice),
                    raw_slice.iloc[env.lookback-1].open,
                    raw_slice.iloc[env.lookback-1].low,
                    raw_slice.iloc[env.lookback-1].high,
                    raw_slice.iloc[env.lookback-1].close,
                    raw_slice.iloc[env.lookback-1].volume,
                ])




pd.DataFrame(episode_targets,columns=['targets','open','low','high','close','volume']).to_csv(env.df_path+'/targets.csv', index=False)

In [8]:
episode_columns = ['targets','close','open','high','low','volume']

#pd.DataFrame([[1,2],[1,2]],columns=['a','b'])
#np.array(episode_targets).shape ,=['targets','close','open','high','low','volume']).
pd.DataFrame(episode_targets,columns=episode_columns)

,targets,close,open,high,low,volume
0,6.936459,44.325001,44.205002,44.860001,42.215191,130756400.0
1,7.254576,44.312500,43.299999,44.325001,41.542770,204420400.0
2,7.467441,43.627499,42.632500,43.737499,40.801189,166116000.0
3,7.780771,42.540001,41.767502,42.540001,40.050072,202561600.0
4,8.421785,41.382500,41.174999,41.842499,39.814007,184192800.0
...,...,...,...,...,...,...
390,19.266792,54.432499,54.389999,55.032501,53.860348,84632400.0
391,19.333063,54.990002,54.779999,55.205002,54.056297,73274800.0
392,19.861558,55.264999,54.860001,55.712502,54.563301,101360000.0
393,18.858794,55.502499,55.092499,55.939999,54.119972,88242400.0


In [9]:
state = env.reset()

In [10]:
state

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  3.21567299e-02,  1.72761590e-03,
         2.56051383e-02,  2.89301766e-03, -4.64018332e-01,
         1.00000000e+00,  1.16400003e-01,  2.98163274e-02,
         1.06514163e+05,  1.69976211e+04,  1.51447543e+04,
         6.77565060e-01,  1.18546900e+00,  5.12920096e-01,
         1.58393589e+04,  1.57717474e+04,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.40000000e-02,  8.51000000e-01,  1.25000000e-01,
         9.98900000e-01,  4.80833333e-01,  7.10000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  7.28165978e-03,  3.28113132e-02,
         1.54072707e-02,  2.92309160e-02, -8.14255244e-02,
         7.50000000e-01,  1.21899996e-01,  6.64801895e-02,
         8.13196126e+02,  1.65677708e+03,  1.50793003e+

In [11]:
next_state = env.step([1])
next_state 

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  7.28165978e-03,  3.28113132e-02,
          1.54072707e-02,  2.92309160e-02, -8.14255244e-02,
          7.50000000e-01,  1.21899996e-01,  6.64801895e-02,
          8.13196126e+02,  1.65677708e+03,  1.50793003e+03,
          6.74054769e-01,  1.18902489e+00,  5.06887924e-01,
          1.56146477e+03,  1.55802240e+03,  6.92727273e-02,
          7.63454545e-01,  1.67272727e-01,  1.51963636e-01,
          6.20909091e-02,  8.41636364e-01,  9.63636364e-02,
          4.95472727e-01,  4.81250000e-01,  6.80000000e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  4.64004293e-02,  5.00676337e-02,
          4.20935955e-02,  5.89101414e-02,  7.25438752e-01,
          5.00000000e-01,  1.31499996e-01,  1.47731083e-01,
          1.29212573e+02,  8.06563263e+

In [12]:
next_state[0][0] == state[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [13]:
# env.reset()
# for i in range(100):
#     next_state, reward, done, _ = env.step(0)
#     print(i, reward,done)#,next_state[-1])
#     if done:
#         break

In [14]:
len(env.df),env.window_size, env.lookback, env.window_size+ env.lookback

(76, 66, 10, 76)

In [15]:
# prices = [5,2,1,7,0,6,3]

# """
# # mins = min de inicio até i inclusive 
#     0 - 5
#     1 - 2
#     2 - 2
#     3 - 2
# # maxs = max de i até fim inclusive
#     0 - 6
#     1 - 6
#     2 - 6
#     3 - 3
# """

# mins = [] #[5,2,2,2]
# maxs = [] #[6,6,6,3]

# ### MINS
# # i = 0 -> prices [0 -> 1] [0:1]
# # mins.append(min(prices[0:1]))
# ### MAXS
# # i = 0 -> prices[0:]
# # i = 1 -> prices[1:]
# # prices[3:]

# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
    
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [16]:
# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
    
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [17]:
# prices = env.df.close.values

# mins = [] 
# maxs = [] 

# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
# #     
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [18]:
#prices * (0.5)
#env.fees['BUY']

In [19]:
# env.n_dataframes

In [20]:
# targets = []
# for i in range(env.n_dataframes):
#     env.load_dataset_by_index(i)
#     target = env.calculate_episode_target(env.df)
#     targets.append(target)
    
# targets

In [21]:
# import pandas as pd
# pd.DataFrame(targets,columns=['targets']).to_csv(env.df_path+'/targets.csv', index=False)

In [22]:
#pd.read_csv(env.df_path+'/targets.csv')

In [23]:
from src.agents.actor_critic.a2c import A2CAgent

In [24]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
    ])
    
    
    return env


In [25]:
agent=A2CAgent(environment,batch_size=5000, epochs=1)
environment().success_threshold

0.12262316637777877

In [26]:
agent.learning_log.episodes

0

In [27]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(timesteps=-1 log_each_n_episodes=100)

Episode * 100 * Worker * False * Moving Avg Reward is ==> 0.00078 * Last Reward was ==> -0.00578



KeyboardInterrupt



In [ ]:
agent.env.mode = "test"
scores = []
success = 0

for i in range(100):
    state = agent.env.reset()
    score = 0
    reward = 0
    done = False
    initial_portfolio = agent.env.portfolio_value
    step = 0
    while not done:
                
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if step < 65:
                success +=1
        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)
    
print(np.mean(scores))
print(min(scores))
print(max(scores))
print(success)